In [1]:
import emoji
import re
import pandas as pd
all_emoji_tweets = pd.read_csv('all_emoji_tweets_test.csv')

In [2]:
# this method was necessary as the Vader model splits text into tokens using whitespace.
# we have added whitespace before and after every emoji:
def add_space_after_emoji(s):
    emojis = " ".join([emoj for emoj in emoji.UNICODE_EMOJI.keys() if len(emoj)==1])
    subbed = re.sub('(?<=[' + emojis +'])', ' ', s)
    subbed = re.sub('(?=[' + emojis +'])', ' ', s)
    
    return subbed

In [3]:
spaced_tweets = []
for tweet in all_emoji_tweets.text:
    spaced_tweet = add_space_after_emoji(tweet)
    spaced_tweets.append(spaced_tweet)
    
all_emoji_tweets['spaced_tweets'] = spaced_tweets

In [4]:
import newVADER
from newVADER.vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [5]:
orig_predictions = []
for tweet in all_emoji_tweets.spaced_tweets:
    score = analyser.polarity_scores(tweet)['compound']
    orig_predictions.append(score)

In [6]:
predictions = [i for i in range(all_emoji_tweets.shape[0])]
for i in range(len(orig_predictions)):
    if orig_predictions[i] > 0.05:
        predictions[i] = 'positive'
    elif orig_predictions[i] < -0.05:
        predictions[i] = 'negative'
    else:
        predictions[i] = 'neutral'

In [7]:
from sklearn.metrics import accuracy_score

print(accuracy_score(all_emoji_tweets.sentiment, predictions))

0.6797385620915033


In [8]:
pd.DataFrame(predictions).to_csv("r_method_2_a12_predictions.csv", index=False, header=False)